In [1]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
# from surprise.model_selection.split import train_test_split
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import json
import os, io
from surprise import KNNBasic, KNNWithMeans
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy
from io import StringIO
from surprise.model_selection import KFold

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/librarythingreviews/reviews.json


In [2]:
def parse(path): 
    g = open(path, 'rb') 
    for l in g: 
        yield eval(l) 
        
def getDF(path):
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

In [3]:
df = getDF('/kaggle/input/librarythingreviews/reviews.json')

In [4]:
df.tail()

,work,flags,unixtime,stars,nhelpful,time,comment,user
1707065,5377722,[],1.376438e+09,3.0,0,"Aug 14, 2013","Yea, so, this is borderline my type of thing. ...",Jellyn
1707066,13302111,[],1.356739e+09,NaN,0,"Dec 29, 2012",solito pacco editoriale natalizio di monologhi...,ShanaPat
1707067,452711,[],1.220227e+09,4.0,0,"Sep 1, 2008",In The Last Dive: A Father and Sons Fatal Desc...,koeniel
1707068,3109878,[],1.195690e+09,NaN,2,"Nov 22, 2007",The Age of Turbulence by Alan Greenspan\nA Rev...,Ductor
1707069,3253,[],1.220227e+09,NaN,0,"Sep 1, 2008",What a fabulous book! The story of a mix up at...,vincent1959


In [5]:
new_df=df.drop(columns=['flags','unixtime','nhelpful','time','comment'])

In [6]:
new_df.head()

,work,stars,user
0,3206242,5.0,van_stef
1,12198649,5.0,dwatson2
2,12533765,NaN,edspicer
3,12981302,4.0,amdrane2
4,5231009,3.0,Lila_Gustavus


In [7]:
new_df.describe()

,stars
count,1.387209e+06
mean,3.805416e+00
std,9.999712e-01
min,5.000000e-01
25%,3.000000e+00
50%,4.000000e+00
75%,4.500000e+00
max,5.000000e+00


In [8]:
df_zero_imputed = new_df.fillna(0)

In [9]:
df_zero_imputed.head()

,work,stars,user
0,3206242,5.0,van_stef
1,12198649,5.0,dwatson2
2,12533765,0.0,edspicer
3,12981302,4.0,amdrane2
4,5231009,3.0,Lila_Gustavus


In [10]:
n_users = len(df_zero_imputed.user.unique())

In [11]:
n_users

83195

In [12]:
n_books = len(df_zero_imputed.work.unique())
n_books

506165

In [13]:
user_list=df_zero_imputed['user'].values.tolist()

In [14]:
from collections import defaultdict
temp = defaultdict(lambda: len(temp))
res = [temp[ele] for ele in user_list]

In [15]:
res

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 13,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 27,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 31,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 94,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181

In [16]:
df_zero_imputed['user_id'] = res

In [17]:
df_zero_imputed.head()

,work,stars,user,user_id
0,3206242,5.0,van_stef,0
1,12198649,5.0,dwatson2,1
2,12533765,0.0,edspicer,2
3,12981302,4.0,amdrane2,3
4,5231009,3.0,Lila_Gustavus,4


In [18]:
ratings_df=df_zero_imputed[['user_id','work','stars']]

In [19]:
ratings_df.head()

,user_id,work,stars
0,0,3206242,5.0
1,1,12198649,5.0
2,2,12533765,0.0
3,3,12981302,4.0
4,4,5231009,3.0


In [20]:
ratings_df=ratings_df.rename(columns={"work": "book_id", "stars": "rating"})

In [21]:
ratings_df.isnull().any()

user_id    False
book_id    False
rating     False
dtype: bool

In [22]:
#create training set
trainingSet, testSet = train_test_split(ratings_df, test_size=0.2, train_size=None, random_state=None, shuffle=True)

In [23]:
trainingSet.shape,testSet.shape,type(trainingSet)

((1365656, 3), (341414, 3), pandas.core.frame.DataFrame)

In [24]:
reader = Reader(rating_scale=(0.5,5))  #invoke reader instance of surprise library
data=Dataset.load_from_df(trainingSet,reader)

In [25]:
test_set = Dataset.load_from_df(testSet, reader=reader)
test_set = test_set.build_full_trainset().build_testset()

# K-fold cross validation

In [26]:
kf = KFold(n_splits=3)
algo = SVD()
for trainset, valset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(valset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.1879
RMSE: 1.1901
RMSE: 1.1863


In [27]:
trainset = data.build_full_trainset()

In [28]:
# Build an algorithm, and train it. Follow methodology provided previously
algo = SVD()
algo.fit(trainset)

In [29]:
predictions = algo.test(test_set)

In [30]:
accuracy.rmse(predictions, verbose=True)

RMSE: 1.1661


1.1661244201285732

# GridSearchCV

In [31]:
param_grid = {'n_epochs': [10,20], 'lr_all': [0.001,0.01,0.1,1],
              'reg_all': [0.1,0.3,1],'n_factors': [20,30]}

In [32]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5) 

In [33]:
gs.fit(data)

In [34]:
print(gs.best_score['rmse'])

1.1551281146571606


In [35]:
print(gs.best_params['rmse'])

{'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1, 'n_factors': 20}


In [36]:
svd = gs.best_estimator['rmse']
svd.fit(data.build_full_trainset())

In [37]:
predictions = svd.test(test_set)

In [38]:
accuracy.rmse(predictions, verbose=True)

RMSE: 1.1439


1.1438602332649164